In [79]:
import numpy as np
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import random

In [80]:
from sklearn.datasets import load_diabetes
regression_data, regression_labels = load_diabetes(return_X_y=True)
regression_data.shape, regression_labels.shape

((442, 10), (442,))

In [81]:
# regression_data[0]

In [82]:
class regression_Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [83]:
class regression_Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
#         prediction = (1/len(self.labels))*sum(self.labels)
        prediction = sum(self.labels)/len(self.labels)
        return prediction   

In [84]:
def regression_avg(labels):
#     print(f"labels = {labels}")
#     print(f"result = {sum(sum(labels))}")
    return sum(labels)/len(labels)

In [85]:
def regression_disp(labels):
    avg_labels = regression_avg(labels)
    return sum((np.array(labels)-avg_labels)**2)/(len(labels)-1)
regression_disp([4,8,2,9])

10.916666666666666

In [86]:
def regression_H(labels):
    
#     print(labels)
#     print(avg_labels)
#     return sum((np.array(labels)-avg_labels)**2)/(len(labels)-1)
#     return (1/len(labels))*regression_disp(labels)
    return regression_disp(labels)
# regression_H([4,8,2,9])

In [87]:
def regression_quality(left_labels, right_labels, current_quality):
    #Расчёт критерия информативности
    #Q(X_m,j,t)
#     print(left_labels)
    p=float(left_labels.shape[0])/(left_labels.shape[0]+right_labels.shape[0])
    return current_quality - p*regression_H(left_labels) - (1-p)*regression_H(right_labels)

In [88]:
def regression_split(data, labels, index, t):
#     print(f"входные data = {data}")
#     print(f"входные labels = {labels}")
#     print(f"index = {index}")
#     print(f"t = {t}")
    left = np.where(data[:, index] <= t)
#     print(f"split . left = {left}")
    right = np.where(data[:, index] > t)
#     print(f"split . right = {right}")
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
#     print(f"true_data={true_data}")
#     print(f"true_labels={true_labels}")
        
    return true_data, false_data, true_labels, false_labels

In [89]:
def regression_find_best_split(data, labels, min_leaf, max_leaf):
    #  обозначим минимальное количество объектов в узле
    # Прежде чем искать лучшее разбиение, мы определяем его текущее состояние
    # Что такое лучшее разбиение?
    current_disp= regression_H(labels)
#     current_quality = 0
#     print(f"Первичный расчёт квалити: {current_quality}")
    best_quality = 0
    best_t = None
    best_index = None
    n_features = [0,1,2,3,5,6,7,8,9]
    for index in n_features: 
        t_values = np.unique([row[index] for row in data])
        
#         print(f"/    Текущий index: {index}")
        for t in t_values: 
            #Перебираем все значение в t_values, используя их как разделители для дерева
            true_data, false_data, true_labels, false_labels = regression_split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
#                 print(f"/        НЕ Достигнут лимит min_leaf={min_leaf}")
                continue
            if len(true_data) > max_leaf or len(false_data) > max_leaf:
#                 print(f"/        Достигнут лимит max_leaf={max_leaf}")
                continue

            current_quality = regression_quality(true_labels, false_labels, current_disp)
#             print(f"/        current_quality = {current_quality}")
                #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
#                 print(f"/        new current_quality({current_quality}) > best_quality({best_quality})")
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index
# regression_find_best_split(regression_data, regression_labels, max_leaf=400)

np.where(regression_data[:, 0] <= -0.107225631607358)

qw = np.where(regression_data[:, 0] <= -0.107225631607358)

qw[0][0]
    

In [90]:
def regression_build_tree(data, labels, min_leaf, max_leaf):
    quality, t, index = regression_find_best_split(data, labels, min_leaf, max_leaf)
#     print(f"regression_build_tree: quality, t, index = {quality, t, index}")
    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0: 
#         print(f"/    Строю лист из следующих данных")
#         print(f"/    возвратные data = {data}")
#         print(f"/    возвратные labels = {labels}")
        return regression_Leaf(data, labels) #Формируем лист из полученных data и labels,
        #иначе
    
    #разбиваем текущее содержимое data и labels с помощью index и t, найденных функцией find_best_split
#     print(f"/    regression_split(data, labels, {index}, {t})")
    true_data, false_data, true_labels, false_labels = regression_split(data, labels, index, t)
    
    # Рекурсивно строим два поддерева
    true_branch = regression_build_tree(true_data, true_labels, min_leaf, max_leaf)
    false_branch = regression_build_tree(false_data, false_labels, min_leaf, max_leaf)
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
#     print("Конец функции build_tree")
    return regression_Node(index, t, true_branch, false_branch)

In [91]:
def regression_predict_something(obj, node):
#     print(f"regression_predict_something")
#     print(f"    Получено на вход obj = {obj}")
#     print(f"    Получено на вход node = {node}")
    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, regression_Leaf):
#         print(f"        node={node}")
#         print(f"regression_Leaf={regression_Leaf.keys}")
        answer = node.prediction
#         print(f"        Ответ:{answer}")
        return answer
    
#     print(f"    node.t={node.t}")
#     print(f"    regression_Leaf={regression_Leaf}")
    
    if obj[node.index] <= node.t:
        return regression_predict_something(obj, node.true_branch)
    else:
        return regression_predict_something(obj, node.false_branch)

In [92]:
def regression_predict(data, tree):
#     print(data)
    
    classes = []
    for obj in data:
        prediction = regression_predict_something(obj, tree)
        classes.append(prediction)
    return classes

In [93]:
# Разобьем выборку на обучающую и тестовую

from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(regression_data, 
                                                                                     regression_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 42)
# classification_labels

In [94]:
my_regression_tree = regression_build_tree(train_data, train_labels, min_leaf=5, max_leaf=400)

In [95]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, regression_Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    
print_tree(my_regression_tree)

Индекс 2
Порог 0.00457216660300077
--> True:
  Индекс 8
  Порог 0.0133159679089277
  --> True:
    Индекс 8
    Порог -0.0439854025655911
    --> True:
      Индекс 5
      Порог -0.0248000120604336
      --> True:
        Индекс 3
        Порог -0.0538708002672419
        --> True:
          Прогноз: 61.714285714285715
        --> False:
          Индекс 1
          Порог -0.044641636506989
          --> True:
            Индекс 5
            Порог -0.0686407967109681
            --> True:
              Прогноз: 96.4
            --> False:
              Прогноз: 146.8
          --> False:
            Индекс 8
            Порог -0.0781409106690696
            --> True:
              Прогноз: 61.4
            --> False:
              Прогноз: 88.16666666666667
      --> False:
        Индекс 6
        Порог 0.0302319104297145
        --> True:
          Прогноз: 47.833333333333336
        --> False:
          Прогноз: 68.125
    --> False:
      Индекс 6
      Порог 0.0228686348215404
 

In [96]:
# Получим ответы для обучающей выборки 
train_answers = regression_predict(train_data, my_regression_tree)
# [-0.14312642 -0.22418983] -2.72879581

In [97]:
# И получим ответы для тестовой выборки
answers = regression_predict(test_data, my_regression_tree)

In [98]:
train_accuracy = r2_score(train_labels, train_answers)
train_accuracy

0.8119276248623002

In [99]:
test_accuracy = r2_score(test_labels, answers)
test_accuracy

0.20990178426289863

In [100]:
test_labels

array([219.,  70., 202., 230., 111.,  84., 242., 272.,  94.,  96.,  94.,
       252.,  99., 297., 135.,  67., 295., 264., 170., 275., 310.,  64.,
       128., 232., 129., 118., 263.,  77.,  48., 107., 140., 113.,  90.,
       164., 180., 233.,  42.,  84., 172.,  63.,  48., 108., 156., 168.,
        90.,  52., 200.,  87.,  90., 258., 136., 158.,  69.,  72., 171.,
        95.,  72., 151., 168.,  60., 122.,  52., 187., 102., 214., 248.,
       181., 110., 140., 202., 101., 222., 281.,  61.,  89.,  91., 186.,
       220., 237., 233.,  68., 190.,  96.,  72., 153.,  98.,  37.,  63.,
       184., 144., 150., 280., 125.,  59.,  65., 281., 277., 167.,  90.,
        72., 178.,  88., 270., 101., 197.,  97.,  53.,  71., 262.,  52.,
       102., 166., 189., 173., 220., 206.,  97.,  60.,  61., 242., 121.,
       128., 104., 265., 132., 283., 174., 129., 257., 137.,  63.,  93.,
       232.])

In [101]:
np.array(answers)

array([183.5       , 234.4       , 183.5       , 267.85714286,
        98.875     ,  79.5       , 225.75      , 267.85714286,
        99.8       , 114.4       ,  64.        , 176.25      ,
       146.8       , 245.16666667,  81.2       , 114.4       ,
       233.42857143, 225.75      , 245.16666667, 179.2       ,
        79.        ,  79.5       ,  68.125     , 216.        ,
       145.88888889, 145.11111111, 234.4       , 173.5       ,
       146.8       ,  98.875     , 225.75      ,  47.83333333,
       204.        , 183.5       , 119.8       , 225.75      ,
        81.2       ,  98.875     ,  90.4       ,  68.125     ,
        68.125     , 104.14285714, 168.6       ,  90.4       ,
       176.25      ,  68.125     ,  61.71428571, 104.14285714,
       146.8       , 143.83333333, 119.8       ,  47.83333333,
       114.4       , 142.4       , 253.8       , 145.88888889,
        50.8       , 216.        ,  50.8       ,  50.8       ,
       221.57142857, 145.88888889, 145.88888889, 104.14